## Formatted Zone

In [2]:
# Importing useful dependencies
import boto3

In [3]:
# Setup S3 client for MinIO (MinIO implements Amazon S3 API)
s3 = boto3.client(
    "s3",
    endpoint_url="http://127.0.0.1:9000", # MinIO API endpoint
    aws_access_key_id="minioadmin", # User name
    aws_secret_access_key="minioadmin", # Password
)

In [4]:
# This function copies all objects from a bucket (under a prefix) into another bucket, creating the destination bucket if it doesn’t exist.
def replicate_bucket(src_bucket, dest_bucket, src_prefix=""):
    # Create destination bucket if it doesn't exist
    existing = s3.list_buckets()
    if dest_bucket not in [b["Name"] for b in existing["Buckets"]]:
        s3.create_bucket(Bucket=dest_bucket)

    paginator = s3.get_paginator("list_objects_v2") # It returns objects in pages and not all at once.
    for page in paginator.paginate(Bucket=src_bucket, Prefix=src_prefix):
        for obj in page.get("Contents", []):

            if obj['Size'] == 0: # skip the folder itself (persistent-landing)
                continue

            key = obj["Key"]

            # Remove the prefix part (persistent-landing) from the key
            new_key = key[len(src_prefix):]

            # Copy object without top-level folder (persistent-landing)
            copy_source = {"Bucket": src_bucket, "Key": key}
            s3.copy_object(Bucket=dest_bucket, Key=new_key, CopySource=copy_source)

            print(f"Copied: {key} -> {new_key}")

In [5]:
# Replicate files from Persistent Landing to Formatted Zone
replicate_bucket(src_bucket = "landing-zone", src_prefix="persistent-landing/", dest_bucket = "formatted-zone")

Copied: persistent-landing/images/image_1759102414626.jpg -> images/image_1759102414626.jpg
Copied: persistent-landing/images/image_1759102414636.jpg -> images/image_1759102414636.jpg
Copied: persistent-landing/images/image_1759102414644.jpg -> images/image_1759102414644.jpg
Copied: persistent-landing/images/image_1759102414652.jpg -> images/image_1759102414652.jpg
Copied: persistent-landing/images/image_1759102414663.jpg -> images/image_1759102414663.jpg
Copied: persistent-landing/images/image_1759102414672.jpg -> images/image_1759102414672.jpg
Copied: persistent-landing/images/image_1759102414681.jpg -> images/image_1759102414681.jpg
Copied: persistent-landing/images/image_1759102414692.jpg -> images/image_1759102414692.jpg
Copied: persistent-landing/images/image_1759102414701.jpg -> images/image_1759102414701.jpg
Copied: persistent-landing/images/image_1759102414711.jpg -> images/image_1759102414711.jpg
Copied: persistent-landing/images/image_1759102414720.jpg -> images/image_175910